# Doc2Vec

## 0. Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
! bash install_mecab-ko_on_colab190912.sh

/content
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 450kB 55.7MB/s 
     |████████████████████████████████| 92kB 14.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-07-11 03:39:13--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::22c3:9b0a

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
import pandas as pd
import nltk, re
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [ ]:
import os
os.chdir('/content/drive/MyDrive/BOAZ_ADV_대법관_김보아즈/data')

In [ ]:
news = pd.read_csv('뉴스/news_prepro.csv')
laws = pd.read_csv('textrank_to_kobart/0706_KoBart512.csv')

In [ ]:
news.head(2)

,category,title,contents
0,교통,"갑툭튀 역주행'에 사망사고도…전동킥보드 사고, 3년 새 8배 늘어",전동킥보드 보급이 늘어나면서 관련 사고도 3년 새 8배 급증한 것으로 나타났다. 3...
1,교통,운전 중 휴대전화 사용 금지… 헌재 전원일치로 합헌 결정,자동차 운전 중 휴대전화 사용을 금지한 법 조항이 헌법에 어긋나지 않는다는 헌법재판...


In [ ]:
laws.head(2)

,Unnamed: 0,category,article_original,abstract,kobart_sum
0,0,사문서위조·사문서위조행사·공정증서원본불실기재·공정증서원본불실기재행사,검사의 상고이유를 본다.원심은 피고인들에 대한 공정증서 원본 불실기재 동행사의 ...,"외국판결에 의하여 신고한 경우 공정증서원본 분실기재, 동 행사죄의 성립을 부정한 사례",피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...
1,1,소유권이전등기,"피고소송대리인들의 상고이유 제1, 2점에 대하여,원판결이 든 증거에 의하여 원판...",대물변제 예약의 법리를 오해하여 부동산의 인도지연으로 인한 입료상당의 손해금의 지...,상고이유 제3점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...


## 1. Data Preprocessing

In [ ]:
# NA 제거
laws.drop(['Unnamed: 0'], axis=1, inplace=True)
laws_na = laws[laws['kobart_sum'].isnull()==False].reset_index(drop=True)

In [ ]:
# 전처리
def text_cleaning(texts):
    string = re.sub(r'[·@%\\*=()/~#&\+á‘’“”?\xc3\xa1\-\|\:\;\!\-\,\_\~\$\'\"\[\]]', ' ', texts) #remove punctuation
    string = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ", string)
    string = re.sub(r'\n',' ', string)    # remove enter
    string = re.sub(r'[0-9]+', '', string)
    string = re.sub(r'\s+', ' ', string)  #remove extra space
    string = re.sub(r'<[^>]+>',' ',string) #remove Html tags
    string = re.sub(r'www.jeju.go.kr',' ',string) #remove url
    string = re.sub(r'jmp',' ',string) #remove url
    string = re.sub(r'index.htm',' ',string) #remove url
    return string

In [ ]:
# 불용어 정의
sw = set(['항소', '법원', '법', '재판'])

In [ ]:
# 토큰화
def text_tokenizing(doc,tokenizer):
  if tokenizer == "words":
    return [word for word in doc.split() if word not in sw]
  elif tokenizer == "nouns":
    return [word for word in mecab.nouns(doc) if word not in sw]
  elif tokenizer == "morphs":
    return [word for word in mecab.morphs(doc) if word not in sw]
  return words

In [ ]:
# 전처리
news['prepro'] = news['contents'].progress_apply(text_cleaning)
# 토큰화
news_data = news['prepro'].tolist()
news['nouns'] = [text_tokenizing(new, tokenizer = "nouns") for new in tqdm(news_data)]


100%|██████████| 30/30 [00:00<00:00, 4555.39it/s]

100%|██████████| 30/30 [00:00<00:00, 210.04it/s]


In [ ]:
# 전처리
laws_na['prepro'] = laws_na['kobart_sum'].progress_apply(text_cleaning)
# 토큰화
laws_data = laws_na['prepro'].tolist()
laws_na['nouns'] = [text_tokenizing(law, tokenizer = "nouns") for law in tqdm(laws_data)]


100%|██████████| 40342/40342 [00:03<00:00, 13408.19it/s]

100%|██████████| 40342/40342 [00:53<00:00, 750.47it/s]


In [ ]:
laws_na.head(2)

,category,article_original,abstract,kobart_sum,prepro,morphs,nouns
0,사문서위조·사문서위조행사·공정증서원본불실기재·공정증서원본불실기재행사,검사의 상고이유를 본다.원심은 피고인들에 대한 공정증서 원본 불실기재 동행사의 ...,"외국판결에 의하여 신고한 경우 공정증서원본 분실기재, 동 행사죄의 성립을 부정한 사례",피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...,피고인들에게들에 대한 공정증서 원본 불실기재 동행사의 점에 관하여는 피고인들이 미국...,"[피고인, 에게, 대한, 공정증서, 원본, 불실, 기재, 동행, 관하, 피고인, 미...","[피고인, 공정증서, 원본, 불실, 기재, 동행, 피고인, 미국, 법원, 확정, 판..."
1,소유권이전등기,"피고소송대리인들의 상고이유 제1, 2점에 대하여,원판결이 든 증거에 의하여 원판...",대물변제 예약의 법리를 오해하여 부동산의 인도지연으로 인한 입료상당의 손해금의 지...,상고이유 제3점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...,상고이유 제 점에 대하여 원판결이 든 증거에 의하여 원판결의 소론 판단 사실을 수긍...,"[상고, 이유, 대하, 원판결, 증거, 의하, 원판결, 소론, 판단, 사실, 수긍,...","[상고, 이유, 원판결, 증거, 원판결, 소론, 판단, 사실, 수긍, 검사, 소론,..."


## 2. Category

In [ ]:
laws_na['category'].value_counts()

손해배상                                          1190
소유권이전등기                                        850
소유권이전등기말소                                      760
손해배상(기)                                        725
거절사정                                           646
                                              ... 
유족보상금지급거부처분취소                                    1
토지분할불가처분취소                                       1
석유및석유대체연료사업법위반·조세범처벌법위반                          1
강도강간미수ㆍ강도상해                                      1
특수공무집행방해등(변경된죄명:특수공무집행방해치상)·폭력행위등처벌에관한법률위반       1
Name: category, Length: 11586, dtype: int64

In [ ]:
def make_category(data):
    data['new_cat'] = ''
    if data['category'].str.contains('교통') | data['category'].str.contains('도로'):
        data['new_cat'] = '도로'
    elif data['category'].str.contains('절도'):
        data['new_cat'] = '절도'
    elif data['category'].str.contains('노동') | data['category'].str.contains('임금') | data['category'].str.contains('근로'):
        data['new_cat'] = '노동 및 임금'
    else: 
        data['new_cat'] = '그 외'
    return data

In [ ]:
law_road = laws_na[laws_na['category'].str.contains('교통')|laws_na['category'].str.contains('도로')].reset_index(drop=True)
law_theft = laws_na[laws_na['category'].str.contains('절도')].reset_index(drop=True)
law_work = laws_na[laws_na['category'].str.contains('노동')|laws_na['category'].str.contains('임금')|laws_na['category'].str.contains('근로')].reset_index(drop=True)

In [ ]:
law_road['category'] = '교통'
law_theft['category'] = '절도'
law_work['category'] = '노동 및 임금'

In [ ]:
law_doc = pd.concat([law_road, law_theft, law_work], axis=0).reset_index(drop=True).reset_index()

In [ ]:
law_doc.category.value_counts()

교통         696
절도         672
노동 및 임금    488
Name: category, dtype: int64

In [ ]:
news_road = news[news['category']=='교통'].reset_index(drop=True)
news_theft = news[news['category']=='절도'].reset_index(drop=True)
news_work = news[news['category']=='노동 및 임금'].reset_index(drop=True)

## 3. Doc2Vec

In [ ]:
law_doc.head(2)

,index,category,article_original,abstract,kobart_sum,prepro,morphs,nouns
0,0,교통,상고이유를 본다.기록에 의하여 살펴보면피고인이 자동차를 운전하였다는 판시 주차장...,나이트크럽의 주차장이 도로교통법상의 도로라고 할 수 없다고 한 사례,피고인이 자동차를 운전하였다는 판시 주차장은 나이트크럽을 출입하는 자들을 위한 작은...,피고인이 자동차를 운전하였다는 판시 주차장은 나이트크럽을 출입하는 자들을 위한 작은...,"[피고인, 자동차, 운전, 다는, 판시, 주차장, 나이트, 출입, 위한, 주차장, ...","[피고인, 자동차, 운전, 판시, 주차장, 나이트, 출입, 주차장, 도로법, 유료도..."
1,1,교통,"검사의 상고이유에 대하여 판단한다.원심은,피고인이 화물자동차를 운전하여 매시 약...",정차 중인 버스를 앞지르기 하던 화물자동차의 왼쪽 일부가 중앙선을 침범한 상태에서...,피고인이 운전하던 화물자동차의 진행차선 내에서 피해자를 화물자동차의 차체 오른쪽부분...,피고인이 운전하던 화물자동차의 진행차선 내에서 피해자를 화물자동차의 차체 오른쪽부분...,"[피고인, 운전, 화물, 자동차, 진행, 차선, 에서, 피해자, 화물, 자동차, 차...","[피고인, 운전, 화물, 자동차, 진행, 차선, 피해자, 화물, 자동차, 차체, 오..."


In [ ]:
doc_df = law_doc[['index','nouns']].values.tolist()
tagged_data = [TaggedDocument(words=words, tags=[id]) for id, words in doc_df]

In [ ]:
max_epochs = 10

model = Doc2Vec(
    window=10,
    size=150,
    alpha=0.025, 
    min_alpha=0.025,
    min_count=2,
    dm =1,
    negative = 5,
    seed = 1234)
  
model.build_vocab(tagged_data)

for epoch in tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).

100%|██████████| 10/10 [00:21<00:00,  2.11s/it]


In [ ]:
news_road.head(2)

,category,title,contents,prepro,morphs,nouns
0,교통,"갑툭튀 역주행'에 사망사고도…전동킥보드 사고, 3년 새 8배 늘어",전동킥보드 보급이 늘어나면서 관련 사고도 3년 새 8배 급증한 것으로 나타났다. 3...,전동킥보드 보급이 늘어나면서 관련 사고도 년 새 배 급증한 것으로 나타났다 일 삼성...,"[전동, 보드, 보급, 늘어나, 면서, 관련, 사고, 급증, 으로, 나타났, 화재,...","[전동, 보드, 보급, 관련, 사고, 급증, 화재, 자사, 접수, 사고, 건수, 분..."
1,교통,운전 중 휴대전화 사용 금지… 헌재 전원일치로 합헌 결정,자동차 운전 중 휴대전화 사용을 금지한 법 조항이 헌법에 어긋나지 않는다는 헌법재판...,자동차 운전 중 휴대전화 사용을 금지한 법 조항이 헌법에 어긋나지 않는다는 헌법재판...,"[자동차, 운전, 휴대, 전화, 사용, 금지, 조항, 헌법, 어긋나, 는다는, 헌법...","[자동차, 운전, 휴대, 전화, 사용, 금지, 조항, 헌법, 헌법, 재판소, 판단,..."


In [ ]:
# 도로
model.random.seed(1234)

for i in range(len(news_road)):
    print(news_road['contents'][i])
    print(news_road['nouns'][i])
    doc_list = news_road['nouns'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    for rd in return_docs:
        for cat, des in zip(law_doc[law_doc['index'] == rd[0]]['category'], law_doc[law_doc['index'] == rd[0]]['kobart_sum']):
            print(rd[0], rd[1], cat, des)
    print('====='*10)

전동킥보드 보급이 늘어나면서 관련 사고도 3년 새 8배 급증한 것으로 나타났다. 3일 삼성화재가 자사에 접수된 사고 건수를 분석한 결과 지난해 접수된 차대 전동킥보드 사고 건수는 1447건으로 2017년 181건에서 8배 이상 급증했다. 관련 피해 금액도 2017년 8억원 정도 수준에서 2020년도는 37억원 정도로 늘었다. 특히 올해 1 5월 접수된 사고 건수가 이미 800여 건에 달해 증가세가 이어질 전망이다. 수도권 10 30대의 남성 전체 사고 비중의 70 이상 차지 지난해 전동킥보드 사고의 특성을 분석해보면 남성이 70 이상이고 10 30대의 사고 비중이 70 이상을 차지했다. 지역별로는 수도권에서 발생하는 비중이 70 이상으로 나타났다. 해당 분석결과 주로 젊은 남성이 도심지역에서 공유 전동킥보드를 많이 이용하면서 사고의 위험에 더 많이 노출된 것으로 해석된다. 사고 중 127건의 사고영상 분석결과 전동킥보드와 차량간의 사고는 전동킥보드의 역주행 신호위반 횡단 중 킥보드 탑승 등 전동킥보드 이용자의 교통법규 미준수에 따른 사고발생 빈도가 높았다. 특히 인도를 주행하다가 이면도로 접속구간 또는 주차장 진출입로를 횡단할 때 발생한 사고 26 와 신호등이 없는 이면도로 교차로에서 서행하지 않은 채 통행하다 발생한 충돌사고 26 가 가장 많았다. 사고영상 분석 127건 중 111건 87.4 은 전동킥보드 이용자가 안전모를 착용하지 않은 상태였다. 도로교통법에 따라 전동킥보드 이용자는 안전모 착용이 필수다. 그러나 전동킥보드 공유서비스 이용자의 안전모 착용은 전적으로 사용자에게 일임되고 있으며 사용 전 안전모 착용 점검이나 안전모 제공 서비스 등이 미흡해 사고 시 피해가 커질 수 있다. 전동킥보드는 구조상 자전거에 비해 바퀴가 작고 이용자의 무게중심이 높기 때문에 급정거 또는 교통사고가 발생했을 경우 이용자가 쉽게 넘어져 두부와 안면부 상해 위험이 상대적으로 높아 안전모 착용이 꼭 필요하다. 전제호 삼성교통안전문화연구소 책임연구원은 최근 전동킥보드 공유서비스

In [ ]:
# 절도
model.random.seed(1234)

for i in range(len(news_theft)):
    print(news_theft['contents'][i])
    doc_list = news_theft['nouns'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    for rd in return_docs:
        for cat, des in zip(law_doc[law_doc['index'] == rd[0]]['category'], law_doc[law_doc['index'] == rd[0]]['kobart_sum']):
            print(rd[0], rd[1], cat, des)
    print('====='*10)

지난해 11월 29일 오전 6시 인천 연수구의 한 빌라. 유명 택배회사 새벽 배송 기사인 A 35 씨가 공동 현관문을 열고 들어가더니 한 집 앞에 놓여 있던 택배 상자를 들고 나왔다. 시가 95만원짜리 아이폰12 128기가 가 들어 있는 상자였다. 그가 이날부터 지난 2월 15일까지 인천시 일대 아파트와 다세대 주택에 몰래 침입해 물품을 훔친 횟수는 모두 54회. 훔친 물건은 시가로 6400여만원어치에 달했다. 고가 물품이 배송된 상자를 훔치려 집 앞까지 갔지만 수취인이 상자를 갖고 들어가 미수에 그친 경우도 5차례나 됐다. A씨는 회사 내부 프로그램에 접속해 동료 기사들이 배송한 물품 내역과 배송지 정보를 확인했다. 특히 자신이 근무하는 새벽 시간에 배송을 마친 휴대전화나 노트북 PC 등 값비싼 배송품을 집중적으로 노렸다. A씨는 배송된 물품이 사라진다 는 피해자 신고를 받은 회사 측이 소속 기사들 차량 GPS 위성항법장치 기록을 대조하는 과정에서 덜미를 잡혔다. 업체 관계자는 기사들이 배송 지원을 하는 경우가 많아 내부 시스템에서 다른 배송 기사의 배송품 정보를 공유하고 있다 고 말했다. 인천지법 형사12단독 강산아 판사는 야간 주거침입 절도 및 야간 주거침입 절도 미수 혐의로 재판에 넘겨진 A씨에게 징역 1년 6개월을 선고했다고 2일 밝혔다. A씨는 2018년 20대 여성을 상대로 스폰 만남 을 빙자해 성관계를 강요하다 6개월 복역했다. 2019년 7월 출소한 뒤 배송 기사가 됐다. A씨는 불법 스포츠 도박으로 4억8000만원의 빚을 지자 배송 물품을 훔쳐 생활비를 마련하기로 하고 범행을 저지른 것으로 드러났다. 재판부는 죄질이 나쁘고 피해 금액이 크며 다른 범죄로 처벌받은 전력도 있어 엄히 처벌할 필요가 있다 고 밝혔다.
442 0.2409476786851883 교통 피고인이 도로관리업무에 계약직으로 근무를 하며 본인 소유의 레간자 승용차를 운전하는 자인바, 피고인 차량 진행방향 좌측 노상에 주차되어 있는 부산31마1886호 라세티 승용차의 우측 휀더 

In [ ]:
# 노동
model.random.seed(1234)

for i in range(len(news_work)):
    print(news_work['contents'][i])
    doc_list = news_work['nouns'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    for rd in return_docs:
        for cat, des in zip(law_doc[law_doc['index'] == rd[0]]['category'], law_doc[law_doc['index'] == rd[0]]['kobart_sum']):
            print(rd[0], rd[1], cat, des)
    print('====='*10)

70대 노동자들이 최저임금에 훨씬 못 미치는 시급을 받고 있다는 조사 결과가 나왔다. 이들 중 일부는 하루 12시간 이상 장시간 노동에 시달리면서도 최저임금조차 받지 못하고 있었다. 노년알바노조 준비위원회와 평등노동자회는 1일 기자회견을 열고 70대 노동자의 최저임금 실태조사 결과를 발표했다. 기초연금을 받는 전국 65세 이상 노동자 45명 청소 18명 경비 15명 돌봄 2명 임명 1명 무직 9명 을 대상으로 지난 5월 18일부터 한 달간 조사한 것이다. 조사 결과 청소 노동자는 10명 중 4명꼴로 근무시간 대비 시급이 최저임금에 미치지 못한다 고 응답했다. 평균 연령 69.3세인 이들의 평균 시급은 8546원으로 올해 최저시급인 8720원보다 적었다. 가장 낮은 시급은 6028원이었다. 하루 근무시간은 10명 중 9명이 6∼7시간이라고 답했다. 고용주가 최저임금 위반을 피하려 의도적으로 노동시간을 줄였다는 게 이들 단체의 주장이다. 법에 보장된 최저임금을 맞추려고 휴식 시간을 늘리거나 노동 강도를 높이는 식으로 꼼수를 썼다는 것이다. 경비 노동자는 실제 근무시간 대비 평균 시급이 6346원으로 파악됐다. 대부분 하루 12시간 맞교대 방식으로 근무하는데 고용주가 실제 근무시간의 3분의 1가량인 4시간을 휴식 시간으로 산정해 최저임금 위반을 피하고 있다고 노동자들은 지적한다. 이들의 평균 연령은 70.1세이고 최고령자는 77세다. 돌봄 노동자 역시 장시간 노동을 하면서도 최저시급 기준에 못 미치는 건 마찬가지다. 간병인 A씨 72 는 업무 특성상 환자와 숙식을 병행하고 밤낮없이 돌봄이 필요해 종일 근무와 대기를 지속하는 상황 이라며 월급을 시급으로 따지면 5000원도 되지 않는다 고 토로했다. 허영구 노년알바노조 준비위원장은 최저임금 위반과 꼼수가 고령층 노동자들에게 만연한 상태 라며 최저임금 금액에만 논의가 집중되다 보니 불법에 대해선 사실상 방치됐다 고 지적했다. 그러면서 노년 노동에 대한 정부 차원의 실태조사와 고용노동부의 정기적인 근로감독을 촉구했다.
1

* 전체

In [ ]:
laws_na = laws_na.reset_index()

In [ ]:
doc_df = laws_na[['index','nouns']].values.tolist()
tagged_data = [TaggedDocument(words=words, tags=[id]) for id, words in doc_df]

In [ ]:
# 10분
max_epochs = 10

model = Doc2Vec(
    window = 10,
    size = 150,
    alpha = 0.025, 
    min_alpha = 0.025,
    min_count = 2,
    dm = 1,
    negative = 5,
    seed = 1234)
  
model.build_vocab(tagged_data)

for epoch in tqdm(range(max_epochs)):
    model.train(tagged_data,
                total_examples = model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).

100%|██████████| 10/10 [08:31<00:00, 51.12s/it]


In [ ]:
# 도로
model.random.seed(1234)

for i in range(len(news_road)):
    print(news_road['contents'][i])
    print(news_road['nouns'][i])
    doc_list = news_road['nouns'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive = [inferred_vector], topn=5)
    for rd in return_docs:
        for cat, des in zip(laws_na[laws_na['index'] == rd[0]]['category'], laws_na[laws_na['index'] == rd[0]]['kobart_sum']):
            print(rd[0], rd[1], cat, des)
    print('====='*10)

전동킥보드 보급이 늘어나면서 관련 사고도 3년 새 8배 급증한 것으로 나타났다. 3일 삼성화재가 자사에 접수된 사고 건수를 분석한 결과 지난해 접수된 차대 전동킥보드 사고 건수는 1447건으로 2017년 181건에서 8배 이상 급증했다. 관련 피해 금액도 2017년 8억원 정도 수준에서 2020년도는 37억원 정도로 늘었다. 특히 올해 1 5월 접수된 사고 건수가 이미 800여 건에 달해 증가세가 이어질 전망이다. 수도권 10 30대의 남성 전체 사고 비중의 70 이상 차지 지난해 전동킥보드 사고의 특성을 분석해보면 남성이 70 이상이고 10 30대의 사고 비중이 70 이상을 차지했다. 지역별로는 수도권에서 발생하는 비중이 70 이상으로 나타났다. 해당 분석결과 주로 젊은 남성이 도심지역에서 공유 전동킥보드를 많이 이용하면서 사고의 위험에 더 많이 노출된 것으로 해석된다. 사고 중 127건의 사고영상 분석결과 전동킥보드와 차량간의 사고는 전동킥보드의 역주행 신호위반 횡단 중 킥보드 탑승 등 전동킥보드 이용자의 교통법규 미준수에 따른 사고발생 빈도가 높았다. 특히 인도를 주행하다가 이면도로 접속구간 또는 주차장 진출입로를 횡단할 때 발생한 사고 26 와 신호등이 없는 이면도로 교차로에서 서행하지 않은 채 통행하다 발생한 충돌사고 26 가 가장 많았다. 사고영상 분석 127건 중 111건 87.4 은 전동킥보드 이용자가 안전모를 착용하지 않은 상태였다. 도로교통법에 따라 전동킥보드 이용자는 안전모 착용이 필수다. 그러나 전동킥보드 공유서비스 이용자의 안전모 착용은 전적으로 사용자에게 일임되고 있으며 사용 전 안전모 착용 점검이나 안전모 제공 서비스 등이 미흡해 사고 시 피해가 커질 수 있다. 전동킥보드는 구조상 자전거에 비해 바퀴가 작고 이용자의 무게중심이 높기 때문에 급정거 또는 교통사고가 발생했을 경우 이용자가 쉽게 넘어져 두부와 안면부 상해 위험이 상대적으로 높아 안전모 착용이 꼭 필요하다. 전제호 삼성교통안전문화연구소 책임연구원은 최근 전동킥보드 공유서비스

In [ ]:
# 절도
model.random.seed(1234)

for i in range(len(news_theft)):
    print(news_theft['contents'][i])
    print(news_theft['nouns'][i])
    doc_list = news_theft['nouns'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive = [inferred_vector], topn=5)
    for rd in return_docs:
        for cat, des in zip(laws_na[laws_na['index'] == rd[0]]['category'], laws_na[laws_na['index'] == rd[0]]['kobart_sum']):
            print(rd[0], rd[1], cat, des)
    print('====='*10)

지난해 11월 29일 오전 6시 인천 연수구의 한 빌라. 유명 택배회사 새벽 배송 기사인 A 35 씨가 공동 현관문을 열고 들어가더니 한 집 앞에 놓여 있던 택배 상자를 들고 나왔다. 시가 95만원짜리 아이폰12 128기가 가 들어 있는 상자였다. 그가 이날부터 지난 2월 15일까지 인천시 일대 아파트와 다세대 주택에 몰래 침입해 물품을 훔친 횟수는 모두 54회. 훔친 물건은 시가로 6400여만원어치에 달했다. 고가 물품이 배송된 상자를 훔치려 집 앞까지 갔지만 수취인이 상자를 갖고 들어가 미수에 그친 경우도 5차례나 됐다. A씨는 회사 내부 프로그램에 접속해 동료 기사들이 배송한 물품 내역과 배송지 정보를 확인했다. 특히 자신이 근무하는 새벽 시간에 배송을 마친 휴대전화나 노트북 PC 등 값비싼 배송품을 집중적으로 노렸다. A씨는 배송된 물품이 사라진다 는 피해자 신고를 받은 회사 측이 소속 기사들 차량 GPS 위성항법장치 기록을 대조하는 과정에서 덜미를 잡혔다. 업체 관계자는 기사들이 배송 지원을 하는 경우가 많아 내부 시스템에서 다른 배송 기사의 배송품 정보를 공유하고 있다 고 말했다. 인천지법 형사12단독 강산아 판사는 야간 주거침입 절도 및 야간 주거침입 절도 미수 혐의로 재판에 넘겨진 A씨에게 징역 1년 6개월을 선고했다고 2일 밝혔다. A씨는 2018년 20대 여성을 상대로 스폰 만남 을 빙자해 성관계를 강요하다 6개월 복역했다. 2019년 7월 출소한 뒤 배송 기사가 됐다. A씨는 불법 스포츠 도박으로 4억8000만원의 빚을 지자 배송 물품을 훔쳐 생활비를 마련하기로 하고 범행을 저지른 것으로 드러났다. 재판부는 죄질이 나쁘고 피해 금액이 크며 다른 범죄로 처벌받은 전력도 있어 엄히 처벌할 필요가 있다 고 밝혔다.
['지난해', '월', '일', '오전', '시', '인천', '연수구', '빌라', '유명', '택배', '회사', '새벽', '배송', '기사', '씨', '공동', '현관문', '집', '앞', '택배', '상자', '시', '만

In [ ]:
# 노동 및 임금
model.random.seed(1234)

for i in range(len(news_work)):
    print(news_work['contents'][i])
    print(news_work['nouns'][i])
    doc_list = news_work['nouns'][i]
    inferred_vector = model.infer_vector(doc_list)
    return_docs = model.docvecs.most_similar(positive=[inferred_vector], topn=5)
    for rd in return_docs:
        for cat, des in zip(laws_na[laws_na['index'] == rd[0]]['category'], laws_na[laws_na['index'] == rd[0]]['kobart_sum']):
            print(rd[0], rd[1], cat, des)
    print('====='*10)

70대 노동자들이 최저임금에 훨씬 못 미치는 시급을 받고 있다는 조사 결과가 나왔다. 이들 중 일부는 하루 12시간 이상 장시간 노동에 시달리면서도 최저임금조차 받지 못하고 있었다. 노년알바노조 준비위원회와 평등노동자회는 1일 기자회견을 열고 70대 노동자의 최저임금 실태조사 결과를 발표했다. 기초연금을 받는 전국 65세 이상 노동자 45명 청소 18명 경비 15명 돌봄 2명 임명 1명 무직 9명 을 대상으로 지난 5월 18일부터 한 달간 조사한 것이다. 조사 결과 청소 노동자는 10명 중 4명꼴로 근무시간 대비 시급이 최저임금에 미치지 못한다 고 응답했다. 평균 연령 69.3세인 이들의 평균 시급은 8546원으로 올해 최저시급인 8720원보다 적었다. 가장 낮은 시급은 6028원이었다. 하루 근무시간은 10명 중 9명이 6∼7시간이라고 답했다. 고용주가 최저임금 위반을 피하려 의도적으로 노동시간을 줄였다는 게 이들 단체의 주장이다. 법에 보장된 최저임금을 맞추려고 휴식 시간을 늘리거나 노동 강도를 높이는 식으로 꼼수를 썼다는 것이다. 경비 노동자는 실제 근무시간 대비 평균 시급이 6346원으로 파악됐다. 대부분 하루 12시간 맞교대 방식으로 근무하는데 고용주가 실제 근무시간의 3분의 1가량인 4시간을 휴식 시간으로 산정해 최저임금 위반을 피하고 있다고 노동자들은 지적한다. 이들의 평균 연령은 70.1세이고 최고령자는 77세다. 돌봄 노동자 역시 장시간 노동을 하면서도 최저시급 기준에 못 미치는 건 마찬가지다. 간병인 A씨 72 는 업무 특성상 환자와 숙식을 병행하고 밤낮없이 돌봄이 필요해 종일 근무와 대기를 지속하는 상황 이라며 월급을 시급으로 따지면 5000원도 되지 않는다 고 토로했다. 허영구 노년알바노조 준비위원장은 최저임금 위반과 꼼수가 고령층 노동자들에게 만연한 상태 라며 최저임금 금액에만 논의가 집중되다 보니 불법에 대해선 사실상 방치됐다 고 지적했다. 그러면서 노년 노동에 대한 정부 차원의 실태조사와 고용노동부의 정기적인 근로감독을 촉구했다.
[